In [1]:
import tensorflow as tf
import gym
import numpy as np

In [2]:
def test_environment(iterations):
    env = gym.make("MountainCar-v0")   
    
    print(env.action_space)
    print(env.observation_space)
    
    env.reset()
    
    for _ in range(iterations):
        env.render()
        env.step(env.action_space.sample()) # take a random action
    
    env.close()

In [3]:
test_environment(10)

C:\Users\fijo\Anaconda3\envs\tf-gpu\lib\site-packages\gym\envs\registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Discrete(3)
Box(2,)


In [4]:
env = gym.make("MountainCar-v0")
env = env.unwrapped
env.seed(1)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


[1]

In [5]:
dims = env.observation_space.shape

observation_size = dims[0]
action_size = env.action_space.n

max_episodes = 10000
learning_rate = 0.01  ##probar diferentes usando tensorflow
gamma = 0.95 #discount rate

In [6]:
def calculate_discounted_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards -mean) / (std)
    
    return discounted_episode_rewards

In [7]:
class PolicyGradient():
    
    def __init__(self):
        self.observations = tf.placeholder(shape=(None,observation_size),dtype = tf.float32, name = "observations")
        self.actions = tf.placeholder(shape = (None, action_size), dtype = tf.float32, name = "actions")
        self.discounted_rewards_ = tf.placeholder(shape = (None,), dtype=tf.float32, name = "discounted_episodes_rewards")

        #usamos esta variable para visualizar en tensorboard
        self.mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

        #3 fully connected layers
        self.fc1 = tf.layers.dense(inputs=self.observations, units=20,activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(),name="Fc1")

        self.fc2 = tf.layers.dense(inputs=self.fc1, units= 10, activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(), name = "Fc2")

        self.fc3 = tf.layers.dense(inputs = self.fc2, units= 5, activation = tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer(), name = "Fc3")

        self.output = tf.layers.dense(inputs=self.fc3, units = 3, activation= None, kernel_initializer=tf.contrib.layers.xavier_initializer(), name = "output")

        self.action_distribution = tf.nn.softmax(self.output, name= "action_distribution")

        self.neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.output, labels = self.actions, name = "sotfmax")
        self.loss = tf.reduce_mean(self.neg_log_prob * self.discounted_rewards_, name = "loss") 
        self.train_opt = tf.train.AdamOptimizer(learning_rate,name="optimizer").minimize(self.loss)

In [8]:
tf.reset_default_graph()

policy = PolicyGradient()

In [9]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tensorboard/pg/1")

## Losses
tf.summary.scalar("Loss", policy.loss)
## Reward mean
tf.summary.scalar("Reward_mean", policy.mean_reward_)

write_op = tf.summary.merge_all()

In [ ]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
#episode data after full simulation before training
episode_states, episode_actions, episode_rewards = [],[],[]

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    for episode in range(max_episodes):
        episodes_rewards_sum = 0
        state = env.reset()
        env.render()
    
        #we simulate the episode until its finished
        while True:
            #conseguimos las probabilidades de las acciones segun la politica
            action_prob_distribution = sess.run(policy.action_distribution, feed_dict = {policy.observations: state.reshape([1,2])})
            action = np.random.choice(range(action_prob_distribution.shape[1]), p = action_prob_distribution.ravel())
            
            #realizamos esas acciones y guardamos la informacion del estado, accion y recompensa en sus respectivos arrays
            new_state, reward, done, info = env.step(action)
            episode_states.append(state)
            
            action_ = np.zeros(action_size)
            action_[action] = 1
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            
            #when we finished the episode we calculate all the reward and calculate the score function
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                allRewards.append(episode_rewards_sum)
                total_rewards = np.sum(allRewards)
                
                mean_reward = np.mean(allRewards)
                maximumRewardRecorded = np.amax(allRewards)
                
                print("======================================")
                print('Episode: ',episode)
                print('Episode reward sum:  ', episode_rewards_sum)
                print('Mean reward: ', mean_reward)
                print('Maximum reward: ',maximumRewardRecorded)
                
                discounted_episode_rewards = calculate_discounted_rewards(episode_rewards)
                
                #feedforward, gradient calculation and backprop
                # Feedforward, gradient and backpropagation
                loss_, _ = sess.run([policy.loss, policy.train_opt], feed_dict={policy.observations: np.vstack(np.array(episode_states)),
                                                                 policy.actions: np.vstack(np.array(episode_actions)),
                                                                 policy.discounted_rewards_: discounted_episode_rewards})
                
 
                                                                 
                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={policy.observations: np.vstack(np.array(episode_states)),
                                                        policy.actions: np.vstack(np.array(episode_actions)),
                                                        policy.discounted_rewards_: discounted_episode_rewards,
                                                        policy.mean_reward_: mean_reward})
                
               
                writer.add_summary(summary, episode)
                writer.flush()
                
                # Reset the transition stores
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            state = new_state

Episode:  0
Episode reward sum:   -48089.0
Mean reward:  -48089.0
Maximum reward:  -48089.0
Episode:  1
Episode reward sum:   -33212.0
Mean reward:  -40650.5
Maximum reward:  -33212.0
Episode:  2
Episode reward sum:   -4132.0
Mean reward:  -28477.666666666668
Maximum reward:  -4132.0
Episode:  3
Episode reward sum:   -12925.0
Mean reward:  -24589.5
Maximum reward:  -4132.0
Episode:  4
Episode reward sum:   -31297.0
Mean reward:  -25931.0
Maximum reward:  -4132.0
Episode:  5
Episode reward sum:   -10011.0
Mean reward:  -23277.666666666668
Maximum reward:  -4132.0
Episode:  6
Episode reward sum:   -2905.0
Mean reward:  -20367.285714285714
Maximum reward:  -2905.0
Episode:  7
Episode reward sum:   -12480.0
Mean reward:  -19381.375
Maximum reward:  -2905.0
Episode:  8
Episode reward sum:   -37106.0
Mean reward:  -21350.777777777777
Maximum reward:  -2905.0
Episode:  9
Episode reward sum:   -10990.0
Mean reward:  -20314.7
Maximum reward:  -2905.0
Episode:  10
Episode reward sum:   -9433.0
M

Episode:  60
Episode reward sum:   -515189.0
Mean reward:  -40583.934426229505
Maximum reward:  -945.0
Episode:  61
Episode reward sum:   -367584.0
Mean reward:  -45858.12903225807
Maximum reward:  -945.0
Episode:  62
Episode reward sum:   -245825.0
Mean reward:  -49032.206349206346
Maximum reward:  -945.0
Episode:  63
Episode reward sum:   -607022.0
Mean reward:  -57750.796875
Maximum reward:  -945.0
Episode:  64
Episode reward sum:   -478072.0
Mean reward:  -64217.276923076926
Maximum reward:  -945.0
Episode:  65
Episode reward sum:   -106309.0
Mean reward:  -64855.030303030304
Maximum reward:  -945.0
Episode:  66
Episode reward sum:   -709018.0
Mean reward:  -74469.40298507463
Maximum reward:  -945.0
Episode:  67
Episode reward sum:   -495117.0
Mean reward:  -80655.39705882352
Maximum reward:  -945.0
Episode:  68
Episode reward sum:   -745035.0
Mean reward:  -90284.08695652174
Maximum reward:  -945.0
Episode:  69
Episode reward sum:   -574545.0
Mean reward:  -97202.1
Maximum reward: